In [ ]:
#!/bin/bash
!kaggle datasets download CooperUnion/anime-recommendations-database

Dataset URL: https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database
License(s): CC0-1.0
 20% 5.00M/25.0M [00:00<00:01, 13.5MB/s]
100% 25.0M/25.0M [00:00<00:00, 57.3MB/s]


In [ ]:
!unzip anime-recommendations-database.zip -d ./data

Archive:  anime-recommendations-database.zip
  inflating: ./data/anime.csv        
  inflating: ./data/rating.csv       


In [ ]:
!pip install torch-geometric
# !pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
# !pip install pyg-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.7 MB/s eta 0:00:00


In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357282 sha256=4584065ff763084f50b3fd3b0cd3cf6aa44ee74ed5bb86663eb0257ef8b355e0
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
import pandas as pd

# Dosyaların yollarını tanımlayın
anime_file = "data/anime.csv"
rating_file = "data/rating.csv"

# Dosyaları pandas ile oku
anime_df = pd.read_csv(anime_file)
rating_df = pd.read_csv(rating_file)

# İlk birkaç satırı kontrol etmek için
print("Anime DataFrame:")
print(anime_df.head())
print("\nRating DataFrame:")
print(rating_df.head())

# Temel analiz: Kayıp veri kontrolü
print("\nAnime dosyasındaki boş veriler:")
print(anime_df.isnull().sum())

print("\nRating dosyasındaki boş veriler:")
print(rating_df.isnull().sum())

# Verileri birleştirme
# Kullanıcının verdiği rating bilgilerini anime detaylarıyla eşleştirelim
merged_df = pd.merge(rating_df, anime_df, on="anime_id", how="left")

# Örnek: İlk birkaç satırı göster
print("\nBirleştirilmiş DataFrame:")
print(merged_df.head())



Anime DataFrame:
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  
2   114262  
3   673572  
4   151266  

Rating DataFrame:
   user_id  anime_id  rating
0        1        20      -1
1        1        24      -1
2        1        79   

In [ ]:
merged_df

,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,1,20,-1,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P...",TV,220,7.81,683297.0
1,1,24,-1,School Rumble,"Comedy, Romance, School, Shounen",TV,26,8.06,178553.0
2,1,79,-1,Shuffle!,"Comedy, Drama, Ecchi, Fantasy, Harem, Magic, R...",TV,24,7.31,158772.0
3,1,226,-1,Elfen Lied,"Action, Drama, Horror, Psychological, Romance,...",TV,13,7.85,623511.0
4,1,241,-1,Girls Bravo: First Season,"Comedy, Ecchi, Fantasy, Harem, Romance, School",TV,11,6.69,84395.0
...,...,...,...,...,...,...,...,...,...
7813732,73515,16512,7,Devil Survivor 2 The Animation,"Action, Demons, Supernatural",TV,13,7.06,101266.0
7813733,73515,17187,9,Ghost in the Shell: Arise - Border:1 Ghost Pain,"Mecha, Police, Psychological, Sci-Fi",Movie,1,7.64,31747.0
7813734,73515,22145,10,Kuroshitsuji: Book of Circus,"Comedy, Demons, Fantasy, Historical, Shounen, ...",TV,10,8.37,122895.0
7813735,73516,790,9,Ergo Proxy,"Mystery, Psychological, Sci-Fi",TV,23,8.03,265005.0


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Dosyaları yükleyelim
anime_df = pd.read_csv("data/anime.csv")
rating_df = pd.read_csv("data/rating.csv")

# -1 olan rating'leri (izlenmiş ama rating verilmemiş) filtreleyelim
rating_df = rating_df[rating_df['rating'] != -1]

# Veriyi inceleyelim
print(rating_df.head())
print(anime_df.head())


     user_id  anime_id  rating
47         1      8074      10
81         1     11617      10
83         1     11757      10
101        1     15451      10
153        2     11771      10
   anime_id                              name  \
0     32281                    Kimi no Na wa.   
1      5114  Fullmetal Alchemist: Brotherhood   
2     28977                          Gintama°   
3      9253                       Steins;Gate   
4      9969                     Gintama&#039;   

                                               genre   type episodes  rating  \
0               Drama, Romance, School, Supernatural  Movie        1    9.37   
1  Action, Adventure, Drama, Fantasy, Magic, Mili...     TV       64    9.26   
2  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.25   
3                                   Sci-Fi, Thriller     TV       24    9.17   
4  Action, Comedy, Historical, Parody, Samurai, S...     TV       51    9.16   

   members  
0   200630  
1   793665  

İçerik Tabanlı Filtreleme (Content Based)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Genre sütununu işleyelim
anime_df['genre'] = anime_df['genre'].fillna("")  # Boş olanları doldur
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(", "))
genre_matrix = count_vectorizer.fit_transform(anime_df['genre'])

# Cosine similarity hesapla
cosine_sim = cosine_similarity(genre_matrix)

# Öneri fonksiyonu
def content_based_recommendation(anime_name, cosine_sim=cosine_sim, df=anime_df):
    # Anime adından indeks bul
    idx = df[df['name'] == anime_name].index[0]

    # İlgili anime için benzerlik puanlarını al
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Puanlara göre sırala
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # İlk 10 benzer animeyi al (kendisi hariç)
    sim_scores = sim_scores[1:11]

    # İndekslerini al
    anime_indices = [i[0] for i in sim_scores]

    # Anime adlarını döndür
    return df.iloc[anime_indices][['name', 'genre', 'rating']]

# Örnek öneri
recommendations = content_based_recommendation("Kimi no Na wa.")
print(recommendations)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


                                                   name  \
5805                        Wind: A Breath of Heart OVA   
6394                       Wind: A Breath of Heart (TV)   
1111              Aura: Maryuuin Kouga Saigo no Tatakai   
208                       Kokoro ga Sakebitagatterunda.   
504   Clannad: After Story - Mou Hitotsu no Sekai, K...   
1201                     Angel Beats!: Another Epilogue   
1435                                         True Tears   
1436                 &quot;Bungaku Shoujo&quot; Memoire   
1494                                           Harmonie   
1631                                Kimikiss Pure Rouge   

                                             genre  rating  
5805          Drama, Romance, School, Supernatural    6.35  
6394          Drama, Romance, School, Supernatural    6.14  
1111  Comedy, Drama, Romance, School, Supernatural    7.67  
208                         Drama, Romance, School    8.32  
504                         Drama, Romance, S

In [ ]:
content_based_recommendation("Kimi no Na wa.")

,name,genre,rating
5805,Wind: A Breath of Heart OVA,"Drama, Romance, School, Supernatural",6.35
6394,Wind: A Breath of Heart (TV),"Drama, Romance, School, Supernatural",6.14
1111,Aura: Maryuuin Kouga Saigo no Tatakai,"Comedy, Drama, Romance, School, Supernatural",7.67
208,Kokoro ga Sakebitagatterunda.,"Drama, Romance, School",8.32
504,"Clannad: After Story - Mou Hitotsu no Sekai, K...","Drama, Romance, School",8.02
1201,Angel Beats!: Another Epilogue,"Drama, School, Supernatural",7.63
1435,True Tears,"Drama, Romance, School",7.55
1436,&quot;Bungaku Shoujo&quot; Memoire,"Drama, Romance, School",7.54
1494,Harmonie,"Drama, School, Supernatural",7.52
1631,Kimikiss Pure Rouge,"Drama, Romance, School",7.48


Ortak Filtreleme (Collaborative Filtering)

In [ ]:
from surprise import SVD
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy

# Surprise formatında veri hazırlama
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Veriyi train-test olarak böl
trainset, testset = train_test_split(data, test_size=0.2)

# SVD algoritması ile modelleme
model = SVD()
model.fit(trainset)

# Tahmin yapma
predictions = model.test(testset)

# RMSE hesaplama
rmse = accuracy.rmse(predictions)
print(f"RMSE: {rmse}")

# Öneri fonksiyonu
def collaborative_recommendation(user_id, model=model, df=anime_df, n=10):
    # Kullanıcının izlediği animeleri al
    watched_anime = rating_df[rating_df['user_id'] == user_id]['anime_id']

    # İzlemediği animeleri filtrele
    not_watched = df[~df['anime_id'].isin(watched_anime)]

    # Tahminleri yap
    not_watched['predicted_rating'] = not_watched['anime_id'].apply(lambda x: model.predict(user_id, x).est)

    # En yüksek tahmini olanları öner
    recommendations = not_watched.sort_values('predicted_rating', ascending=False).head(n)
    return recommendations[['name', 'genre', 'predicted_rating']]

# Örnek öneri
user_recommendations = collaborative_recommendation(1)
print(user_recommendations)


RMSE: 1.1340
RMSE: 1.1340298615903985
                                                  name  \
0                                       Kimi no Na wa.   
11                                      Koe no Katachi   
830                                       Guilty Crown   
10                                Clannad: After Story   
3                                          Steins;Gate   
57   Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...   
702                                            Another   
175                             Katekyo Hitman Reborn!   
118                                    No Game No Life   
144                      Higurashi no Naku Koro ni Kai   

                                                 genre  predicted_rating  
0                 Drama, Romance, School, Supernatural          9.865383  
11                              Drama, School, Shounen          9.800429  
830                 Action, Drama, Sci-Fi, Super Power          9.770919  
10   Drama, Fantasy, Ro

<ipython-input-14-76fa88dda3bc>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_watched['predicted_rating'] = not_watched['anime_id'].apply(lambda x: model.predict(user_id, x).est)


In [ ]:
user_recommendations

,name,genre,predicted_rating
0,Kimi no Na wa.,"Drama, Romance, School, Supernatural",9.865383
11,Koe no Katachi,"Drama, School, Shounen",9.800429
830,Guilty Crown,"Action, Drama, Sci-Fi, Super Power",9.770919
10,Clannad: After Story,"Drama, Fantasy, Romance, Slice of Life, Supern...",9.731563
3,Steins;Gate,"Sci-Fi, Thriller",9.691565
57,Ano Hi Mita Hana no Namae wo Bokutachi wa Mada...,"Drama, Slice of Life, Supernatural",9.679800
702,Another,"Horror, Mystery, School, Supernatural, Thriller",9.669115
175,Katekyo Hitman Reborn!,"Action, Comedy, Shounen, Super Power",9.641160
118,No Game No Life,"Adventure, Comedy, Ecchi, Fantasy, Game, Super...",9.641047
144,Higurashi no Naku Koro ni Kai,"Mystery, Psychological, Supernatural, Thriller",9.576582


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Genre'leri işleme al
count_vectorizer = CountVectorizer(tokenizer=lambda x: x.split(", "))
genre_matrix = count_vectorizer.fit_transform(anime_df['genre'])

# Cosine benzerliği hesapla
content_sim_matrix = cosine_similarity(genre_matrix)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
from surprise import SVD
from surprise.model_selection import train_test_split

# Surprise formatına çevir
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(rating_df[['user_id', 'anime_id', 'rating']], reader)

# Veriyi train-test olarak böl
trainset, testset = train_test_split(data, test_size=0.2)

# SVD modelini eğit
svd_model = SVD()
svd_model.fit(trainset)


In [ ]:
def hybrid_recommendation(user_id, anime_name, n=10):
    """
    Hybrid öneri sistemi.
    İçerik tabanlı ve collaborative filtering yaklaşımlarını birleştirir.
    """
    # İçerik tabanlı öneriler
    anime_idx = anime_df[anime_df['name'] == anime_name].index[0]
    content_scores = list(enumerate(content_sim_matrix[anime_idx]))
    content_scores = sorted(content_scores, key=lambda x: x[1], reverse=True)
    content_recommendations = [anime_df.iloc[i[0]] for i in content_scores[1:n+1]]

    # Collaborative filtering önerileri
    user_watched = rating_df[rating_df['user_id'] == user_id]['anime_id'].tolist()
    collaborative_recommendations = []

    for anime_id in anime_df['anime_id']:
        if anime_id not in user_watched:
            pred_rating = svd_model.predict(user_id, anime_id).est
            collaborative_recommendations.append((anime_id, pred_rating))

    # Collaborative filtering sonuçlarını sırala
    collaborative_recommendations = sorted(collaborative_recommendations, key=lambda x: x[1], reverse=True)[:n]
    collaborative_recommendations = [anime_df[anime_df['anime_id'] == rec[0]] for rec in collaborative_recommendations]

    # Karma öneri
    hybrid_recommendations = []
    for content, collab in zip(content_recommendations, collaborative_recommendations):
        hybrid_recommendations.append((content['name'], content['genre'], "Content-Based"))
        hybrid_recommendations.append((collab.iloc[0]['name'], collab.iloc[0]['genre'], "Collaborative"))

    return pd.DataFrame(hybrid_recommendations, columns=["Anime", "Genre", "Source"])

# Örnek kullanım
recommendations = hybrid_recommendation(user_id=1, anime_name="Kimi no Na wa.", n=5)
print(recommendations)


                                               Anime  \
0                        Wind: A Breath of Heart OVA   
1                                         Fairy Tail   
2                       Wind: A Breath of Heart (TV)   
3                                             Bleach   
4              Aura: Maryuuin Kouga Saigo no Tatakai   
5                            Shigatsu wa Kimi no Uso   
6                      Kokoro ga Sakebitagatterunda.   
7                              Kaichou wa Maid-sama!   
8  Clannad: After Story - Mou Hitotsu no Sekai, K...   
9                                     Koe no Katachi   

                                               Genre         Source  
0               Drama, Romance, School, Supernatural  Content-Based  
1  Action, Adventure, Comedy, Fantasy, Magic, Sho...  Collaborative  
2               Drama, Romance, School, Supernatural  Content-Based  
3  Action, Comedy, Shounen, Super Power, Supernat...  Collaborative  
4       Comedy, Drama, Romance, S

In [ ]:
hybrid_recommendation(user_id=1, anime_name="Kimi no Na wa.", n=5)

,Anime,Genre,Source
0,Wind: A Breath of Heart OVA,"Drama, Romance, School, Supernatural",Content-Based
1,Fairy Tail,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",Collaborative
2,Wind: A Breath of Heart (TV),"Drama, Romance, School, Supernatural",Content-Based
3,Bleach,"Action, Comedy, Shounen, Super Power, Supernat...",Collaborative
4,Aura: Maryuuin Kouga Saigo no Tatakai,"Comedy, Drama, Romance, School, Supernatural",Content-Based
5,Shigatsu wa Kimi no Uso,"Drama, Music, Romance, School, Shounen",Collaborative
6,Kokoro ga Sakebitagatterunda.,"Drama, Romance, School",Content-Based
7,Kaichou wa Maid-sama!,"Comedy, Romance, School, Shoujo",Collaborative
8,"Clannad: After Story - Mou Hitotsu no Sekai, K...","Drama, Romance, School",Content-Based
9,Koe no Katachi,"Drama, School, Shounen",Collaborative


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

# Dosyaları yükle
anime_df = pd.read_csv("data/anime.csv")
rating_df = pd.read_csv("data/rating.csv")

# -1 rating'leri temizle (izlenmiş ama rating verilmemiş)
rating_df = rating_df[rating_df['rating'] != -1]

# Kullanıcı ve anime ID'lerini indeks olarak dönüştür
user_ids = rating_df['user_id'].unique()
anime_ids = rating_df['anime_id'].unique()

user_id_map = {uid: idx for idx, uid in enumerate(user_ids)}
anime_id_map = {aid: idx for idx, aid in enumerate(anime_ids)}

# ID'leri indekslere çevir
rating_df['user_idx'] = rating_df['user_id'].map(user_id_map)
rating_df['anime_idx'] = rating_df['anime_id'].map(anime_id_map)

# Eğitim ve test setlerini ayır
train, test = train_test_split(rating_df, test_size=0.2, random_state=42)


Derin Öğrenme Modeli: Neural Collaborative Filtering (NCF)

In [ ]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate

# Model parametreleri
num_users = len(user_ids)
num_animes = len(anime_ids)
embedding_size = 50

# Kullanıcı ve anime embedding katmanları
user_input = Input(shape=(1,), name="user_input")
user_embedding = Embedding(num_users, embedding_size, name="user_embedding")(user_input)
user_vec = Flatten(name="user_vec")(user_embedding)

anime_input = Input(shape=(1,), name="anime_input")
anime_embedding = Embedding(num_animes, embedding_size, name="anime_embedding")(anime_input)
anime_vec = Flatten(name="anime_vec")(anime_embedding)

# Vektörleri birleştir
concat = Concatenate()([user_vec, anime_vec])

# Yoğun katmanlar
dense_1 = Dense(128, activation="relu")(concat)
dense_2 = Dense(64, activation="relu")(dense_1)
output = Dense(1, activation="linear")(dense_2)

# Modeli derle
model = Model(inputs=[user_input, anime_input], outputs=output)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Model özeti
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ user_input (InputLayer)   │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ anime_input (InputLayer)  │ (None, 1)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_embedding            │ (None, 1, 50)          │      3,480,000 │ user_input[0][0]       │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ anime_embedding           │ (None, 1, 50)          │        496,350 │ anime_input[0][0]      │
│ (Embedding)               │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ user_vec (Flatten)        │ (None, 50)             │              0 │ user_embedding[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ anime_vec (Flatten)       │ (None, 50)             │              0 │ anime_embedding[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 100)            │              0 │ user_vec[0][0],        │
│                           │                        │                │ anime_vec[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 128)            │         12,928 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 64)             │          8,256 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 1)              │             65 │ dense_1[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,997,599 (15.25 MB)

 Trainable params: 3,997,599 (15.25 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Eğitim verisini hazırlama
x_train = [train['user_idx'].values, train['anime_idx'].values]
y_train = train['rating'].values

x_test = [test['user_idx'].values, test['anime_idx'].values]
y_test = test['rating'].values

# Modeli eğit
history = model.fit(
    x=x_train,
    y=y_train,
    validation_data=(x_test, y_test),
    epochs=10,
    batch_size=1024,
    verbose=1
)


Epoch 1/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 16s 3ms/step - loss: 4.0778 - mae: 1.2868 - val_loss: 1.4153 - val_mae: 0.9045
Epoch 2/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.3670 - mae: 0.8861 - val_loss: 1.3631 - val_mae: 0.8802
Epoch 3/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.2805 - mae: 0.8554 - val_loss: 1.3336 - val_mae: 0.8717
Epoch 4/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.1930 - mae: 0.8230 - val_loss: 1.3340 - val_mae: 0.8647
Epoch 5/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.1016 - mae: 0.7885 - val_loss: 1.3115 - val_mae: 0.8650
Epoch 6/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 1.0221 - mae: 0.7572 - val_loss: 1.3193 - val_mae: 0.8615
Epoch 7/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 0.9524 - mae: 0.7299 - val_loss: 1.3265 - val_mae: 0.8649
Epoch 8/10
4951/4951 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - loss: 0.8938 - mae: 0.7064 - val_loss: 1.3542 - val_mae: 0.8684
Epoch 9/10
4951/4951 ━━━━━━━━━━━

In [ ]:
def recommend_anime(user_id, model, top_n=10):
    """
    Kullanıcıya önerilen animeleri döndür.
    """
    user_idx = user_id_map[user_id]
    all_anime_indices = np.arange(num_animes)

    # Kullanıcı için tahmin yap
    user_array = np.array([user_idx] * num_animes)
    predictions = model.predict([user_array, all_anime_indices], verbose=0)

    # En yüksek tahmini al
    recommended_indices = predictions.flatten().argsort()[-top_n:][::-1]
    recommended_anime_ids = [anime_ids[idx] for idx in recommended_indices]

    # Önerilen anime adlarını döndür
    recommendations = anime_df[anime_df['anime_id'].isin(recommended_anime_ids)]
    return recommendations[['name', 'genre', 'rating']]

# Örnek kullanım
recommendations = recommend_anime(user_id=1, model=model, top_n=5)
print(recommendations)


                          name  \
288                 Fairy Tail   
724        High School DxD New   
804           Sword Art Online   
1057           High School DxD   
1263  Sekirei: Pure Engagement   

                                                  genre  rating  
288   Action, Adventure, Comedy, Fantasy, Magic, Sho...    8.22  
724   Action, Comedy, Demons, Ecchi, Harem, Romance,...    7.87  
804           Action, Adventure, Fantasy, Game, Romance    7.83  
1057      Comedy, Demons, Ecchi, Harem, Romance, School    7.70  
1263          Action, Comedy, Ecchi, Harem, Super Power    7.61  


In [ ]:
recommend_anime(user_id=1, model=model, top_n=5)

,name,genre,rating
288,Fairy Tail,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",8.22
724,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,...",7.87
804,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance",7.83
1057,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School",7.70
1263,Sekirei: Pure Engagement,"Action, Comedy, Ecchi, Harem, Super Power",7.61


# AutoEncoder Collaborative Filtering

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Model
from keras.layers import Input, Dense, Dropout

# Kullanıcı-Anime Etkileşim Matrisini Oluşturma
rating_df = pd.read_csv("data/rating.csv")
anime_df = pd.read_csv("data/anime.csv")

# -1 olan rating'leri sıfırla (izlenmiş ama puan verilmemiş)
rating_df = rating_df[rating_df['rating'] != -1]
rating_df = rating_df.groupby(['user_id', 'anime_id'], as_index=False).agg({'rating': 'mean'})

# Kullanıcı ve anime matrisini oluştur
user_anime_matrix = rating_df.pivot(index='user_id', columns='anime_id', values='rating').fillna(0)
matrix = user_anime_matrix.values

# Eğitim ve test setlerini ayır
train, test = train_test_split(matrix, test_size=0.2, random_state=42)

# Autoencoder Modeli
input_dim = train.shape[1]

input_layer = Input(shape=(input_dim,))
encoder = Dense(128, activation="relu")(input_layer)
encoder = Dropout(0.2)(encoder)
encoder = Dense(64, activation="relu")(encoder)

decoder = Dense(128, activation="relu")(encoder)
decoder = Dense(input_dim, activation="sigmoid")(decoder)

autoencoder = Model(inputs=input_layer, outputs=decoder)
autoencoder.compile(optimizer="adam", loss="mse")

# Modeli Eğit
history = autoencoder.fit(
    train, train,
    epochs=50,
    batch_size=128,
    validation_data=(test, test),
    verbose=1
)

# Öneri Fonksiyonu
def recommend_animes(user_id, user_anime_matrix, autoencoder, top_n=10):
    # Kullanıcı vektörünü al
    user_idx = user_id - 1  # Matris 0 tabanlı olduğu için
    user_vector = user_anime_matrix.iloc[user_idx].values.reshape(1, -1)

    # Modelden tahmin al
    predicted_ratings = autoencoder.predict(user_vector)[0]

    # Kullanıcının daha önce izlemediği animeleri seç
    watched_animes = user_anime_matrix.columns[user_anime_matrix.iloc[user_idx] > 0]
    recommendations = {
        anime_id: predicted_ratings[i] for i, anime_id in enumerate(user_anime_matrix.columns)
        if anime_id not in watched_animes
    }

    # En yüksek puanlı önerileri al
    top_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:top_n]
    recommended_animes = anime_df[anime_df['anime_id'].isin([rec[0] for rec in top_recommendations])]
    return recommended_animes[['name', 'genre', 'rating']]



Epoch 1/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 0.5520 - val_loss: 0.5277
Epoch 2/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5168 - val_loss: 0.5129
Epoch 3/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.5064 - val_loss: 0.5064
Epoch 4/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4917 - val_loss: 0.5004
Epoch 5/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4881 - val_loss: 0.4956
Epoch 6/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4905 - val_loss: 0.4938
Epoch 7/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4855 - val_loss: 0.4930
Epoch 8/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4835 - val_loss: 0.4924
Epoch 9/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4877 - val_loss: 0.4919
Epoch 10/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4873 - val_loss: 0.4915
Epoch 11/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.4836 - val_loss: 0.4913
Epoch 12/50
435/435 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/ste

In [ ]:
# Kullanıcı için öneriler
recommendations = recommend_animes(user_id=2, user_anime_matrix=user_anime_matrix, autoencoder=autoencoder, top_n=5)
print(recommendations)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
                             name  \
40                     Death Note   
72    Kuroko no Basket 2nd Season   
445              Mirai Nikki (TV)   
804              Sword Art Online   
1115                        Free!   

                                                  genre  rating  
40    Mystery, Police, Psychological, Supernatural, ...    8.71  
72                      Comedy, School, Shounen, Sports    8.58  
445   Action, Mystery, Psychological, Shounen, Super...    8.07  
804           Action, Adventure, Fantasy, Game, Romance    7.83  
1115              Comedy, School, Slice of Life, Sports    7.67  
